In [1]:
# based on: https://www.kaggle.com/mlwhiz/multiclass-text-classification-pytorch/notebook
import random
import copy
import time
import pandas as pd
import numpy as np
import gc
import re
import torch

#import spacy
from tqdm import tqdm_notebook, tnrange
from tqdm.auto import tqdm

tqdm.pandas(desc='Progress')
from collections import Counter

from nltk import word_tokenize

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from torch.autograd import Variable
from sklearn.metrics import f1_score
import os 

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

# cross validation and metrics
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
from torch.optim.optimizer import Optimizer

from sklearn.preprocessing import StandardScaler
from multiprocessing import  Pool
from functools import partial
import numpy as np
from sklearn.decomposition import PCA
import torch as t
import torch.nn as nn
import torch.nn.functional as F

import matplotlib.pyplot as plt

In [2]:
embed_size = 300 # how big is each word vector
n_layers = 1
hidden_size = 128
fcl_size = 64
max_features = 200000 # how many unique words to use (i.e num rows in embedding vector)
maxlen = 60 # max number of words in a question to use
batch_size = 512 # how many samples to process at once
n_epochs = 10 # how many times to iterate over all samples

In [3]:
review_text = "It's the best movie I've ever seen!!! <3"

In [4]:
contraction_dict = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not", "didn't": "did not",  "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not", "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",  "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would", "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would", "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam", "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have", "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as", "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would", "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have", "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",  "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have", "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have","you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have", "you're": "you are", "you've": "you have"}
def _get_contractions(contraction_dict):
    contraction_re = re.compile('(%s)' % '|'.join(contraction_dict.keys()))
    return contraction_dict, contraction_re
contractions, contractions_re = _get_contractions(contraction_dict)
def replace_contractions(text):
    def replace(match):
        return contractions[match.group(0)]
    return contractions_re.sub(replace, text)

In [10]:
import re
import string

# remove emojis: https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
def deEmojify(text):
    regrex_pattern = re.compile(pattern = "["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags = re.UNICODE)
    return regrex_pattern.sub(r'',text)

# data preprocessing
def preprocess_text(text):
    
    # converting text to lowercase
    text = text.lower()
    # stop_words = set(stopwords.words('english'))
    text = replace_contractions(text)
    # replacing url-s with the word 'url'
    text = re.sub('((www\.[^\s]+)|(https?://[^\s]+))','', text)
    # replacing usernames-s with the word 'user'
    text = re.sub('@[^\s]+','', text)
    # remove HTML tags
    text = re.sub('<.*?>', '', text)
    # remove punctuation marks
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    # remove multiple spaces
    text = re.sub(' +',' ', text)
    # replacing numbers with the word 'number'
    text = re.sub(r'\w*\d+\w*', '', text)
    
    tokens = word_tokenize(text)
    
    # cleaned_text = [w for w in tokens if not w.lower() in stop_words]
    
    cleaned_text = [w for w in tokens if len(w)>=2]

    # text = deEmojify(text)
    
    return cleaned_text

In [57]:
text_prep = preprocess_text(review_text)
print(f"Preprocessed and tokenized input: {text_prep}")

Preprocessed and tokenized input: ['it', 'is', 'the', 'best', 'movie', 'have', 'ever', 'seen']


In [55]:
text_prep = [text_prep]

Pre-processed and tokenized input: [[['it', 'is', 'the', 'best', 'movie', 'have', 'ever', 'seen']]]


In [13]:
import pickle
# loading tokenizer
with open('input/tokenizer_with_stoppwords.pkl', 'rb') as handle:
    tokenizer = pickle.load(handle)

In [25]:
test_X = tokenizer.texts_to_sequences(list(text_prep))

## Pad the sentences 
test_X = pad_sequences(test_X, maxlen=maxlen)

In [27]:
test_X.shape

(1, 60)

In [28]:
class LSTM(nn.Module):
    
    def __init__(self):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        drp = 0.2
        n_classes = 3
        self.embedding = nn.Embedding(max_features, embed_size)
        # self.embedding.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        # self.embedding.weight.requires_grad = False
        self.lstm = nn.LSTM(embed_size, self.hidden_size, num_layers = n_layers, batch_first=True)
        # self.linear = nn.Linear(self.hidden_size , fcl_size)
        # self.relu = nn.ReLU()
        self.dropout = nn.Dropout(drp)
        self.out = nn.Linear(self.hidden_size, n_classes)


    def forward(self, x):
        h_embedding = self.embedding(x)
        #_embedding = torch.squeeze(torch.unsqueeze(h_embedding, 0))
        h_lstm, _ = self.lstm(h_embedding)
        
        # conc = self.relu(self.linear(h_lstm[:,-1,:]))
        conc = h_lstm[:,-1,:]
        conc = self.dropout(conc)
        out = self.out(conc)
        return out

In [58]:
print("Predicting sentiment...")

Predicting sentiment...


In [29]:
model = LSTM()

In [30]:
model.load_state_dict(torch.load("trained_models/lstm_glove200k_10epoch_with_stoppwords"))
model.cuda()

LSTM(
  (embedding): Embedding(200000, 300)
  (lstm): LSTM(300, 128, batch_first=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (out): Linear(in_features=128, out_features=3, bias=True)
)

In [37]:
inp = torch.LongTensor(test_X).cuda()

In [38]:
inp

tensor([[  0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   0,
           0,   0,   0,   0,   0,   0,   0,   0,   0,   0,   6,   5,   1, 114,
          14,  19, 159, 134]], device='cuda:0')

In [42]:
model.eval()  
y_pred = model(inp).detach()

In [43]:
# keep/store predictions
pred =F.softmax(y_pred, dim = 1).cpu().numpy()

In [48]:
sentiments = ['negative', 'neutral', 'positive']

In [50]:
prob = np.max(pred)
sentiment = sentiments[np.argmax(pred, axis = 1)[0]]

In [51]:
prob

0.9730513

In [52]:
sentiment

'positive'

In [54]:
print(f'The sentence is {int(prob*100)} % likely to be {sentiment}.')

The sentence is 97 % likely to be positive.
